# Merging datasets

In [4]:
# setup
import pandas as pd
import numpy as np

df1 = pd.DataFrame(np.arange(16).reshape(4,4), columns=list("ABCD"))
df2 = pd.DataFrame(np.zeros((3,4)), columns=list("ABCD"))
df1

,A,B,C,D
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15


In [9]:
np.zeros([3,4])

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [5]:
df2

,A,B,C,D
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


## Concat
- concetenates along a particular axis
- set logic

In [6]:
pd.concat([df1, df2])
# df1 and df2 have the same columns name
# index are same as other file

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0


In [11]:
pd.concat([df1, df2]).reset_index(drop=True)

,A,B,C,D
0,0.0,1.0,2.0,3.0
1,4.0,5.0,6.0,7.0
2,8.0,9.0,10.0,11.0
3,12.0,13.0,14.0,15.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0


In [10]:
pd.concat([df1, df2], axis=1)
# df1 shape (4,4), df2 shape (3,4)

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0
3,12,13,14,15,NaN,NaN,NaN,NaN


In [18]:
pd.concat([df1, df2], axis=1, join="inner")

,A,B,C,D,A,B,C,D
0,0,1,2,3,0.0,0.0,0.0,0.0
1,4,5,6,7,0.0,0.0,0.0,0.0
2,8,9,10,11,0.0,0.0,0.0,0.0


## Merge
- different join operations similar to relational 
